In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
restrict_48_hours = True # Option to restrict time series prediction to 48 hours
include_text_embeddings = False
standard_scale_variables = False

In [4]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")


In [5]:
ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_new.pkl"))
ireg_vitals_ts_df = ireg_vitals_ts_df.drop(columns=["hosp_time_delta"])
ireg_vitals_ts_df.rename(columns={'icu_time_delta': 'timedelta'}, inplace=True)

# imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_vitals_icu.pkl"))

# ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))

# if restrict_48_hours:
#     ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['hosp_time_delta'] <= 48]
#     ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['hosp_time_delta'] >= 0]

if restrict_48_hours:
    ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] <= 48]
    ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] >= 0]

In [6]:
print(ireg_vitals_ts_df.shape)

(4811876, 34)


In [7]:
# notes_df = pd.read_pickle(os.path.join(output_dir, "notes_text.pkl"))
notes_df = pd.read_pickle(os.path.join(output_dir, "icu_notes_text_embeddings.pkl"))
notes_df = notes_df[notes_df['stay_id'].notnull()]

if restrict_48_hours:
    notes_df = notes_df[notes_df['icu_time_delta'] <= 48]
    notes_df = notes_df[notes_df['icu_time_delta'] >= 0]

In [8]:
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

if restrict_48_hours:
    icustays_df = icustays_df[icustays_df['los'] >= 2]

In [9]:
valid_stay_ids = icustays_df['stay_id'].unique()

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(valid_stay_ids)]
notes_df = notes_df[notes_df['stay_id'].isin(valid_stay_ids)]

In [10]:

def calculate_avg_embedding(notes_df):
    # For each stay, calculate the average embedding of all notes
    avg_embedding = notes_df.groupby('stay_id')['biobert_embeddings'].apply(lambda x: np.mean(np.vstack(x), axis=0))
    avg_embedding = np.stack(avg_embedding.values, axis=0)

    # Reshape the array to 2 dimensions
    avg_embedding = avg_embedding.reshape(avg_embedding.shape[0], -1)

    embedding_cols = ['emb_{}'.format(i) for i in range(avg_embedding.shape[1])]
    avg_embedding_df = pd.DataFrame(avg_embedding, columns=embedding_cols)

    # notes_df = pd.concat([notes_df.drop('biobert_embeddings', axis=1), avg_embedding_df], axis=1)
    stays = notes_df['stay_id'].unique()
    notes_df = pd.DataFrame({'stay_id': stays})

    notes_df = pd.merge(notes_df, avg_embedding_df, left_index=True, right_index=True)
    return notes_df, embedding_cols

notes_df, embedding_cols = calculate_avg_embedding(notes_df)

In [11]:
admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"))
admissions_df = admissions_df.rename(columns={"hospital_expire_flag": "died"})
admissions_df = admissions_df[["subject_id", "hadm_id", "died"]]

In [13]:
ts_stays = ireg_vitals_ts_df['stay_id'].unique()
notes_stays = notes_df['stay_id'].unique()

unique_stays = np.intersect1d(ts_stays, notes_stays)
print(f"Number of stays with both notes and vitals: {len(unique_stays)}")
print(f"Number of stays with vitals: {len(np.setdiff1d(unique_stays, ts_stays))}")
print(f"Number of stays with notes: {len(np.setdiff1d(unique_stays, notes_stays))}")

Number of stays with both notes and vitals: 32040
Number of stays with vitals: 0
Number of stays with notes: 0


In [14]:
len(ts_stays), len(notes_stays), len(unique_stays)

(35131, 32040, 32040)

In [12]:
ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(unique_stays)].copy()
notes_df = notes_df[notes_df['stay_id'].isin(unique_stays)].copy()

event_list = ireg_vitals_ts_df.columns
event_list = event_list.drop(['subject_id', 'hadm_id', 'stay_id', 'timedelta'])

In [13]:
from sklearn.model_selection import train_test_split

pkl_list = ireg_vitals_ts_df['stay_id'].unique().tolist()

seed = 0
train_id, test_id = train_test_split(pkl_list, test_size=0.3, random_state=seed)

In [14]:
print(f"Number of training stays: {len(train_id)}")
print(f"Number of testing stays: {len(test_id)}")

Number of training stays: 22428
Number of testing stays: 9612


In [15]:
train_ireg_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(train_id)].copy()

cols = train_ireg_ts_df.columns.tolist()
cols = [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'timedelta']]

if standard_scale_variables:
    for col in cols:
        scaler = StandardScaler()
        scaler.fit(train_ireg_ts_df[[col]])
        ireg_vitals_ts_df[col] = scaler.transform(ireg_vitals_ts_df[[col]])


In [16]:
from scipy.signal import find_peaks
from tqdm import tqdm

unique_stays = ireg_vitals_ts_df['stay_id'].unique()

curr_stay = unique_stays[0]

haim_ts_embeddings = pd.DataFrame(columns=['subject_id', 'hadm_id', 'stay_id'])

died = np.zeros(len(unique_stays))
for i in tqdm(range(len(unique_stays)), desc="Embedding TS", total=len(unique_stays)):
    stay = unique_stays[i]
    curr_ts = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == stay]

    curr_subject = curr_ts['subject_id'].iloc[0]
    curr_hadm = curr_ts['hadm_id'].iloc[0]

    row_data = {'subject_id': curr_subject, 'hadm_id': curr_hadm, 'stay_id': stay}

    for event in event_list:
        series = curr_ts[event].dropna() #dropna rows
        if len(series) >0: #if there is any event
            row_data[event+'_max'] = series.max()
            row_data[event+'_min'] = series.min()
            row_data[event+'_mean'] = series.mean(skipna=True)
            row_data[event+'_variance'] = series.var(skipna=True)
            row_data[event+'_meandiff'] = series.diff().mean() #average change
            row_data[event+'_meanabsdiff'] =series.diff().abs().mean()
            row_data[event+'_maxdiff'] = series.diff().abs().max()
            row_data[event+'_sumabsdiff'] =series.diff().abs().sum()
            row_data[event+'_diff'] = series.iloc[-1]-series.iloc[0]
            
            #Compute the n_peaks
            peaks,_ = find_peaks(series)
            row_data[event+'_npeaks'] = len(peaks)
            
            #Compute the trend (linear slope)
            if len(series)>1:
                row_data[event+'_trend']= np.polyfit(np.arange(len(series)), series, 1)[0] #fit deg-1 poly
            else:
                row_data[event+'_trend'] = 0

    haim_ts_embeddings = pd.concat([haim_ts_embeddings, pd.DataFrame(row_data, index=[0])], ignore_index=True)

    died[i] = admissions_df[admissions_df['hadm_id'] == curr_hadm]['died'].iloc[0]

haim_ts_embeddings.fillna(0, inplace=True)

Embedding TS: 100%|██████████| 32040/32040 [1:24:31<00:00,  6.32it/s]


In [17]:
if include_text_embeddings:
    haim_ts_embeddings = haim_ts_embeddings.merge(notes_df[['stay_id'] + embedding_cols], on='stay_id', how='left')

In [18]:
from sklearn.model_selection import train_test_split

pkl_list = haim_ts_embeddings['stay_id'].unique().tolist()

df = haim_ts_embeddings.copy()
df['died'] = died

seed = 0
df = df[~df.isna().any(axis=1)]

# non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()
# df.drop(non_numeric_cols, axis=1, inplace=True)

train_idx = df[df['stay_id'].isin(train_id)]['stay_id'].tolist()
test_idx = df[df['stay_id'].isin(test_id)]['stay_id'].tolist()

y_train = df[df['stay_id'].isin(train_idx)]['died']
y_test = df[df['stay_id'].isin(test_idx)]['died']

x_train = df[df['stay_id'].isin(train_idx)]
x_test = df[df['stay_id'].isin(test_idx)]

x_train.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'died'], inplace=True)
x_test.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'died'], inplace=True)

<ipython-input-18-59da13a5431f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'died'], inplace=True)
<ipython-input-18-59da13a5431f>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'died'], inplace=True)


In [19]:
from prediction_util import run_xgb

y_pred_test, y_pred_prob_test, y_pred_train, y_pred_prob_train, xgb = run_xgb(x_train, y_train, x_test)

In [26]:
print("Number of data in training set: ", len(y_train))
print("Number of data in testing set: ", len(y_test))

Number of data in training set:  22428
Number of data in testing set:  9612


In [21]:
from sklearn import metrics

f1_train = metrics.f1_score(y_train, y_pred_train)
accu_train = metrics.accuracy_score(y_train, y_pred_train)
accu_bl_train = metrics.balanced_accuracy_score(y_train, y_pred_train)
auc_train =  metrics.roc_auc_score(y_train, y_pred_prob_train)
conf_matrix_train = metrics.confusion_matrix(y_train, y_pred_train)

In [22]:
print(f'F1 Score for Training Set is: {f1_train}')
print(f'Accuracy for Training Set is: {accu_train}')
print(f'Balanced Accuracy for Training Set is: {accu_bl_train}')
print(f'AUC for Training Set is: {auc_train}')
print(f'Confusion Matrix for Training Set is: {conf_matrix_train}')

F1 Score for Training Set is: 0.7407668783815574
Accuracy for Training Set is: 0.9017299803816657
Balanced Accuracy for Training Set is: 0.9203341897247759
AUC for Training Set is: 0.9769563110038819
Confusion Matrix for Training Set is: [[17075  2027]
 [  177  3149]]


In [23]:
f1_test = metrics.f1_score(y_test, y_pred_test)
accu_test = metrics.accuracy_score(y_test, y_pred_test)
accu_bl_test = metrics.balanced_accuracy_score(y_test, y_pred_test)
auc_test =  metrics.roc_auc_score(y_test, y_pred_prob_test)
auprc_test = metrics.average_precision_score(y_test, y_pred_prob_test)
conf_matrix_test = metrics.confusion_matrix(y_test, y_pred_test)

In [24]:
print(f'F1 Score for Testing Set is: {f1_test}')
print(f'Accuracy for Testing Set is: {accu_test}')
print(f'Balanced Accuracy for Testing Set is: {accu_bl_test}')
print(f'AUC for Testing Set is: {auc_test}')
print(f'AUPRC for Testing Set is: {auprc_test}')
print(f'Confusion Matrix for Testing Set is: {conf_matrix_test}')

F1 Score for Testing Set is: 0.5246795745841287
Accuracy for Testing Set is: 0.8186641697877652
Balanced Accuracy for Testing Set is: 0.7569905153999265
AUC for Testing Set is: 0.8530120521903127
AUPRC for Testing Set is: 0.5566171796587702
Confusion Matrix for Testing Set is: [[6907 1267]
 [ 476  962]]


In [25]:
# Find the most important features
import matplotlib.pyplot as plt
import seaborn as sns

feature_importance = pd.DataFrame({'feature': x_train.columns, 'importance': xgb.feature_importances_})
feature_importance = feature_importance.sort_values(by='importance', ascending=False)
feature_importance = feature_importance.reset_index(drop=True)

plt.figure(figsize=(10, 10))
sns.barplot(x="importance", y="feature", data=feature_importance)
plt.title('Feature Importance')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "feature_importance.png"), dpi=300)
plt.show()

AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [ ]:
feature_importance